# 한개의 퍼셉트론을 사용한 단일층 회귀 신경망

단일층이란 은닉층이 없이 **입력층과 출력층**만 존재하는 신경망이다.

In [1]:
import numpy as np
import csv

In [2]:
np.random.seed(123)

In [3]:
# 난수 생성을 위한 숫자 설정(가중치 초기값)
RND_MEAN = 0
RND_STD = 0.0030

# 학습률 설정
LEARNING_RATE = 0.001

In [4]:
def main_exec(epoch_count = 10, mb_size = 10, report = 1, train_rate = 0.7):
    load_dataset()                                            # 데이터를 불러오는 함수
    init_model()                                              # 모델 초기화 함수
    train_and_test(epoch_count, mb_size, report, train_rate)  # 학습하는 및 신경망 테스트 함수

In [5]:
# 데이터를 불러오고, 원-핫 인코딩
def load_dataset():
    with open('./Data/abalone.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)    # 헤더를 뛰어넘기기 위해 next를 통해 1번째 행을건너뜀
        rows = []
        for row in csvreader:
            rows.append(row)
            
            
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 10, 1    # 독립변수(X)의 크기와 종속변수(Y)의 크기
    # 버퍼(플레이스홀더처럼 변수를 넣기 위한 임의의 공간)
    data = np.zeros([len(rows), input_cnt + output_cnt])
    
    # 원핫 인코딩 후 나머지는 그대로 옮겨담는 반복문
    for n, row in enumerate(rows):
        if row[0] == 'I': data[n, 0] = 1
        if row[0] == 'M': data[n, 1] = 1
        if row[0] == 'F': data[n, 2] = 1
        data[n, 3:] = row[1:]

In [6]:
# 두 번째 기능 구축하기 - 파라미터 초기화
def init_model():
    global weight, bias, input_cnt, output_cnt
    
    # 하나의 퍼셉트론만 있는 단층 퍼셉트론이므로 가중치는 (10, 1)의 크기를 가짐
    # 편향의 경우는 가중치의 경우도 같은 이유로 (1,1)의 크기
    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt])
    bias = np.zeros([output_cnt])

미니배치 경사 하강법은 확률적 경사하강법과 배치 경사 하강법의 장점을 절충한 방법이다.<br>
확률적 경사하강법을 이용한다면 계산 비용은 적은 대신 가중치가 최적값에 수렴하는 모습이 불안정하고, 배치 경사하강법을 이용하면 이 반대이다.<br>
따라서 미니배치의 크기를 적절하게 맞춰주는 것이 양쪽 모두의 장점을 가져갈 수 있으므로 튜닝의 대상이 된다.

In [31]:
def train_and_test(epoch_count, mb_size, report, train_rate):
    '''
    미니배치 경사하강법을 이용함
    '''
    # 변환하는 값은 미니배치가 몇 덩어리(스텝)으로 쪼개지는지
    step_count = arrange_data(mb_size, train_rate) 

    # 테스트 데이터에 대한 독립변수와 종속변수 반환
    test_x, test_y = get_test_data()                   
    
    # 2중반복문으로 미니배치 -> 에폭 
    # 에폭이 돌아가는 첫번째 반복문
    for epoch in range(epoch_count):
        # 전체 미니배치의 결과(1에폭)인 손실과 정확도를 받는 빈 리스트                     
        losses, accs = [], []
        # 한번의 에폭이 돌아가는 동안 미니배치 스텝카운트 반복 수행
        for n in range(step_count):
            # 미니배치 사이즈와 미니배치의 총 갯수(스텝)의 개별값을 받아 학습데이터의 독립, 종속변수 반환                       
            train_x, train_y = get_train_data(mb_size, n) 
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)
            
        if report > 0 and (epoch+1) % report == 0:
            acc = run_test(test_x, test_y)
            print("Epoch {}: Train - loss = {:5.3f}, accuracy = {:5.3f} / Test = {:5.3f}".format(epoch+1, np.mean(losses), np.mean(accs), acc))
            
    final_acc = run_test(test_x, test_y)
    print("\n최종 테스트 결과: final_accuracy = {:5.3f}".format(final_acc))

In [8]:
def arrange_data(mb_size, train_rate):    # 데이터를 섞어주는 기능 -> 미니배치 단위 출력
    '''
    데이터를 섞어주고 미니배치 단위를 출력해주는 역할을 한다.
    '''
    global data, shuffle_map, test_begin_index
    # arange는 range를 numpy에서 적용한것과 같아 연속된 숫자들을 반환한다.
    # 데이터를 무작위 셔플
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    # 미니배치 스텝 카운트를 구하는 과정
    # 미니배치를 설정하고 남은 데이터들은 테스트 데이터로 사용한다.
    # 스텝을 간단하게 생각하면 미니배치가 몇개로 나누어지는지이다.
    step_count = int(data.shape[0] * train_rate) // mb_size
    
    # 테스트 셋 경계선 탐색
    test_begin_index = step_count * mb_size
    
    return step_count

훈련세트의 샘플을 섞으면 가중치 최적값의 탐색 과정이 가중치 최적값의 탐색 과정이 다양해져 가중치 최적값을 더 제대로 찾을 수 있다.<br>
따라서 에폭마다 샘플을 섞어줄 수 있도록 한다.

In [9]:
# 학습 데이터 분할 과정 기능 구축
def get_train_data(mb_size, nth):
    global shuffle_map, test_begin_index, output_cnt
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_index])
    
    train_data = data[shuffle_map[mb_size * nth : mb_size * (nth+1)]]
    
    return train_data[:,:-output_cnt], train_data[:, -output_cnt:]

# 테스트 데이터 분할 과정 기능 구축(변수 구분 포함)
def get_test_data():
    global data, shuffle_map, test_begin_index, output_cnt
    # 전에 얻은 테스트셋의 경계선을 이용하여 테스트셋을 생성한다.
    test_data = data[shuffle_map[test_begin_index:]]
    
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]

순전파 연산은 $f_\theta(x) = \theta_0 + \theta_{1}x$의 1차식으로 나타낼 수 있다.<br>
($Y=XW+B$의 형태와 동일하다고 생각하면 됨)

In [10]:
# 신경망의 순전파 연산 과정
def forward_neuralnet(x):
    global weight, bias
    output = np.matmul(x, weight) + bias
    # 두 번째 반환값인 x는 aux_nn으로 반환처리 되며, 역전파를 수행할 때 사용할 예정
    return output, x

In [11]:
# MSE를 구하는 함수
def forward_postproc(output, y):
    # 편차 -> 제곱 -> 평균(손실)
    diff = output - y
    square = np.square(diff)
    loss = np.mean(square)
    
    # 두 번째 반환값인 편차는 aux_pp로 반환되며, 역전파를 수행하는데 있어 필요하다
    return loss, diff

In [29]:
# 신경망(회귀)의 평가지표를 산출하는 공식(정확도)
def eval_accuracy(output, y):
    mdiff = np.mean(np.abs((output-y) /y))
    
    return 1 - mdiff

$Y = XW + B$ 라고 식을 두고 진행한다고 하면 연쇄 법칙을 이용하여 다음처럼 진행시킬 수 있다.<br>
**A** &emsp; $\frac{\partial L}{\partial W}=\frac{\partial L}{\partial Y}\frac{\partial Y}{\partial W}$<br><br>
&emsp;&emsp; **A - 1** &ensp; $\frac{\partial Y}{\partial W_{kj}} = \frac{\partial}{\partial W_{kj}}(X_{ik}W_{kj}+B_j) = X_{ik}$ &emsp;&ensp; $(\ast Y= XW + B)$ <br><br>
&emsp;&emsp; **A - 2** &ensp; $\frac{\partial L}{\partial Y_{ij}} = G_{ij}$ &emsp;&ensp; # ---- $G_{ij}$로 가정<br><br>
**B** &emsp; $\frac{\partial L}{\partial Y}\frac{\partial Y}{\partial W} = G_{ij}X_{ik}$<br><br>
&emsp;&emsp; **B - 1** &ensp; $G_{ij}X_{ik} = G_{1j}X_{1k}+G_{2j}X_{2k}+G_{3j}X_{3k}+\cdots + G_{nj}X_{nk}$<br><br>
&emsp;&emsp; **B - 2** &ensp; $\frac{\partial Y}{\partial W_{kj}} = T_{k1}G_{1j}+T_{k2}G_{2j}+T_{k3}G_{3j}+\cdots +T_{km}G_{mj}$ &emsp;&ensp; $(\ast T = X^T)$ &emsp;&ensp; # ---- $X$를 $X^T$전치하여 모양을 맞춤<br><br>
**C** &ensp; $\frac{\partial L}{\partial W} = TG = X^T G$ &emsp;&ensp; # ---- 다시 T를 X로 변환하여 식을 정리<br><br><br>

결과적으로 가중치의 손실 기울기는 $X^T G$로 정리할 수 있고, 이때 $G_{ij}$는 $Y_{ij}$성분에 대한 손실 함수의 기울기로, G_output이라는 변수로 저장한다.

In [19]:
def backprop_postproc(diff):
    shape = diff.shape
    G_loss = 1
    
    g_loss_square = np.ones(shape) / np.prod(shape)
    g_square_diff = 2 * diff
    g_diff_output = 1
    
    G_square = g_loss_square * G_loss
    G_diff = g_square_diff * G_square
    G_output = g_diff_output * G_diff
    
    return G_output

In [23]:
def backprop_neuralnet(G_output, x):
    global weight, bias
    g_output_w = x.transpose()
    
    G_w = np.matmul(g_output_w, G_output)
    G_b = np.sum(G_output, axis = 0)
    
    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

In [17]:
def run_train(x, y):
    output, aux_nn = forward_neuralnet(x)         # 일반적인 신경망의 연산
    loss, aux_pp = forward_postproc(output, y)    # aux는 보조자료 혹은 참고자료라는 의미로 많이 씀
    accuracy = eval_accuracy(output, y)           # 학습의 1단계 (순전파 및 학습 과정)
    
    # 학습의 2단계
    G_loss = 1.0
    G_output = backprop_postproc(G_loss, aux_pp)
    backprop_neuralnet(G_output, aux_nn)
    
    return loss, accuracy

# 평가를 수행하는 함수
def run_test(x, y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    
    return accuracy